In [1]:
import bisect
import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PreprocessFcns import *
import scipy
from scipy.fftpack import fft
from scipy.signal import butter, filtfilt, find_peaks
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
import time
%matplotlib inline

Using TensorFlow backend.


In [2]:
path = r'//FS2.smpp.local\\RTO\\CIS-PD Study\MJFF Curation\Finalized Dataset'
dest = r'//FS2.smpp.local\\RTO\\CIS-PD Study\Watch Features Data'

In [3]:
grouper = LeaveOneGroupOut()
sns.set(font_scale = 1.5)

In [ ]:
VisitNumber = {
    '2 Weeks: Time 0'   : 0,
    '2 Weeks: Time 30'  : 1,
    '2 Weeks: Time 60'  : 2,
    '2 Weeks: Time 90'  : 3,
    '2 Weeks: Time 120' : 4,
    '2 Weeks: Time 150' : 5,
    '1 Month'           : 6
}

ClinicTasks = {
    'Stndg'    : 'Standing',
    'Wlkg'     : 'Walking',
    'WlkgCnt'  : 'Walking while counting',
    'FtnR'     : 'Finger to nose--right hand',
    'FtnL'     : 'Finger to nose--left hand',
    'RamR'     : 'Alternating right hand movements',
    'RamL'     : 'Alternating left hand movements',
    'SitStand' : 'Sit to stand',
    'Drwg'     : 'Drawing on a paper',
    'Typg'     : 'Typing on a computer keyboard',
    'NtsBts'   : 'Assembling nuts and bolts',
    'Drnkg'    : 'Taking a glass of water and drinking',
    'Sheets'   : 'Organizing sheets in a folder',
    'Fldg'     : 'Folding towels',
    'Sitng'    : 'Sitting'
}

In [ ]:
def filterMetaData():
'''filter metadata file to remove empty data and unnecessary scores
add necessary information including binary tremor scores

tasks: list of tasks for which to retrieve metadata of'''

    # open metadata containing scores for each symptom for each task completed
    metaDataFull = pd.read_csv(os.path.join(path, 'Metadata Tables', 'Table4.csv'))
#     # isolate metadata corresponding to tasks of interest specified
#     indices = (x for x in range(len(metaDataFull)) if metaDataFull.TaskAbb.values[x] in tasks)
#     metaDataFull = metaDataFull.loc[indices]

    SubjID = []
    Visit = []
    TaskAbb = []
    AccFile = []
    Tremor = []
    for record in metaDataFull.iterrows():
        # eliminate rows of metadata that contain nan values
        if (type(record[1]['Side']) == float):
            continue
        if (np.isnan(record[1]['Tremor - ' + record[1]['Side']])):
            continue
        # build file name of the recording related to each piece of metadata
        filename = (str(int(record[1]['SubjID'])) + '_' +
                            str(VisitNumber[record[1]['Visit']]) + '_' + 
                            record[1]['TaskAbb'] + '.csv')
        # add file name to file path for easy access
        filepath = os.path.join(path, 'TaskAcc', filename)
        # test is the recording file exists (not all metadata has related acceleration recording)
        if not os.path.exists(filepath):
            continue
        SubjID = SubjID + [int(record[1]['SubjID'])]
        Visit = Visit + [VisitNumber[record[1]['Visit']]]
        TaskAbb = TaskAbb + [record[1]['TaskAbb']]
        AccFile = AccFile + [filename]
        # only concerned with tremor score on the side of subject wearing the apple watch
        Tremor = Tremor + [int(record[1]['Tremor - ' + record[1]['Side']])]
    # create column with binary tremor scores (symptomatic vs normal)
    TremorBIN = [int(t > 0) for t in Tremor]
    metaData = pd.DataFrame({'SubjID': SubjID, 
                             'Visit': Visit, 
                             'TaskAbb': TaskAbb,
                             'AccFile': AccFile,
                             'Tremor': Tremor,
                             'TremorBIN': TremorBIN})
    print('Records = ' + str(len(metaData)))
    
    return metaData

metaData = filterMetaData()

Records = 2014


In [10]:
def DetectDiscernAnomalyNN(tasks, segment, binary, anomaly, metaData):
    '''follows two-step architecture to first detect anomaly in data and filter accordingly
    predictions made on clips that are extrapolated to the scale of the recordings for validaiton
    
    tasks: list of tasks to be considered (Sitting or Standing ideally)
    binary: True/False - whether or not all scores should be considered or just looking for presence of symptom
    anomaly: whether or not to detect and filter by anomaly
    metaData: variable containing relevant metadata generated from the function above'''
    
    # isolate only metadata corresponding to tasks of interest
    indices = (x for x in range(len(metaData)) if metaData.TaskAbb.values[x] in tasks)
    metaData = metaData.loc[indices]
    
    if binary:
        # consider only binay scores (symptomatic - 1 or normal - 0)
        tremScore = 'TremorBIN'
    else:
        # consider all scores on the UPDRS
        tremScore = 'Tremor'
    
    # set threshold for RMSE of acceleration magnitude at which to consider an anomaly
    NormRMSE = 0.01
    
    Data = []
    Anomalies = []
    Labels = []
    Subjects = []
    Recordings = []
    
    for record in metaData.iterrows():
        # get the acceleration from the recording corresponding to each item of metadata in the filtered metadata
        recording = pd.read_csv(os.path.join(path, 'TaskAcc', record[1]['AccFile']), 
                                parse_dates = ['timestamp'])[['timestamp', 'x', 'y', 'z']]
        recording.columns = ['Timestamp', 'X', 'Y', 'Z']
        # calculate magnitude of metadata without filtering the raw data
        recording['Mag'] = np.sqrt((recording.X**2 + recording.Y**2 + recording.Z**2))
        recording = recording.sort_values(by = 'Timestamp', axis = 0)
        
        if segment:
            # group data points in the recording according to intervals
            # change interval by changing the modulo number to half of window length
            recording['TimeWdw'] = [(tm - datetime.timedelta(minutes = 0,
                                                             seconds = tm.second % 2.5,
                                                             microseconds = tm.microsecond)) 
                                    for tm in recording.Timestamp]
            # organize the data by index of epoch time value from first data point in recording
            recording['TimeIdx'] = (recording.Timestamp.values - 
                                    recording.Timestamp.values[0]).astype('timedelta64[ms]').astype(int)
            recording = recording.set_index('TimeIdx')
            for t in recording.TimeWdw.unique():
                # segment the recording into clips grouped by interval with 50% overlap
                # change length of window by changing the value in the timedelta to half of desired
                clip = recording.loc[(recording.TimeWdw == t) | 
                                     (recording.TimeWdw == (t + np.timedelta64(2500, 'ms')))]
                # discard clips significantly different in length from desired time length (50Hz - 5 sec is 250)
                if len(clip) < 200:
                    continue
                    
                if anomaly:
                    # begin generation of an array classifying each segment as anomalous or not for future filtering
                    if np.sqrt(np.mean((clip.Mag - np.mean(clip.Mag))**2)) < NormRMSE:
                        Anomalies = Anomalies + [0]
                    else:
                        Anomalies = Anomalies + [1]
                        
                # upsample the three axes of each clip to normalize length and increase resolution
                fx = scipy.interpolate.interp1d(range(len(clip)), clip.X.values)
                fy = scipy.interpolate.interp1d(range(len(clip)), clip.Y.values)
                fz = scipy.interpolate.interp1d(range(len(clip)), clip.Z.values)
                clipX = fx(np.linspace(start = 0, stop = len(clip) - 1, num = 500))
                clipY = fy(np.linspace(start = 0, stop = len(clip) - 1, num = 500))
                clipZ = fz(np.linspace(start = 0, stop = len(clip) - 1, num = 500))
                # reshape upsampled data into a rectangular form that can be inputted into the neural network
                datasteps = []
                for dpx, dpy, dpz in zip(clipX, clipY, clipZ):
                    datasteps = datasteps + [[dpx, dpy, dpz]]
                Data = Data + [datasteps]
                # add relevant metadata to correlate to the acceleration data being formatted
                Labels = Labels + [record[1][tremScore]]
                Subjects = Subjects + [record[1]['SubjID']]
                Recordings = Recordings + [str(int(record[1]['SubjID'])) + '_' + 
                                           str(int(record[1]['Visit'])) + '_' +
                                           record[1]['TaskAbb']]
        
        # if recordings are to be considered as a whole by the neural network as opposed to clips
        else:
            if anomaly:
                # classify whole recording as anomaly according to threshold of RMSE
                if np.sqrt(np.mean((recording.Mag - np.mean(recording.Mag))**2)) < NormRMSE:
                    Anomalies = Anomalies + [0]
                else:
                    Anomalies = Anomalies + [1]
                    
            # upsample full recordings to 1000 points for normalization and formatting
            fx = scipy.interpolate.interp1d(range(len(recording)), recording.X.values)
            fy = scipy.interpolate.interp1d(range(len(recording)), recording.Y.values)
            fz = scipy.interpolate.interp1d(range(len(recording)), recording.Z.values)
            clipX = fx(np.linspace(start = 0, stop = len(recording) - 1, num = 1000))
            clipY = fy(np.linspace(start = 0, stop = len(recording) - 1, num = 1000))
            clipZ = fz(np.linspace(start = 0, stop = len(recording) - 1, num = 1000))
            # reshape upsampled data into a rectangular form that can be inputted into the neural network
            datasteps = []
            for dpx, dpy, dpz in zip(clipX, clipY, clipZ):
                datasteps = datasteps + [[dpx, dpy, dpz]]
            Data = Data + [datasteps]
            # add relevant metadata to correltate to teh acceleration data being formatted
            Labels = Labels + [record[1][tremScore]]
            Subjects = Subjects + [record[1]['SubjID']]
            Recordings = Recordings + [str(int(record[1]['SubjID'])) + '_' + 
                                       str(int(record[1]['Visit'])) + '_' +
                                       record[1]['TaskAbb']]

    # convert all generated data and metadata to arrays to be used by the neural network
    Data = np.array(Data)
    Anomalies = np.array(Anomalies)
    Labels = np.array(Labels)
    subjects = np.array(Subjects)
    recordings = np.array(Recordings)

    print('(Samples, Timesteps, Features (Axes)) = ' + str(Data.shape))
    print('Labels = ' + str(len(Labels)))
    
    if anomaly:
        # filter all data to separate anomalous data for fitting model and other data for bypassing model and prediction
        Recordings = recordings[Anomalies == 0]
        Subjects = subjects[Anomalies == 0]
        Scores = Labels[Anomalies == 0]
        Predictions = [0] * len(Recordings)
        Densities = [0] * len(Recordings)
        
        Data = Data[Anomalies == 1]
        Labels = Labels[Anomalies == 1]
        subjects = subjects[Anomalies == 1]
        recordings = recordings[Anomalies == 1]
        
    else:
        Recordings = []
        Subjects = []
        Scores = []
        Predictions = []
        Densities = []

    TestLabs = []
    PredLabs = []
    testInds = []
    # split dataset by subject to validation on each subject's data separately (leave-one-subject-out method)
    for trainInd, testInd in grouper.split(Data, Labels, groups = subjects):

        TrainData = Data[trainInd]
        TrainLab = Labels[trainInd]
        TestData = Data[testInd]

        TestLab = Labels[testInd]

        # initialize sequential neural network
        model = Sequential()
        # add a long short term memory layer specifying the input shape (length of 500 (segmented) with 3 axes)
        model.add(LSTM(50, input_shape = (500, 3)))
        # add a dense layer with a sigmoid activation function
        model.add(Dense(1, activation = 'sigmoid'))
        # compile the neural network with a mae loss function and adam optimizer
        model.compile(loss = 'mae', optimizer = 'adam')
        history = model.fit(TrainData, TrainLab, epochs = 10, batch_size = int(len(trainInd) / 20), 
                            validation_data = (TestData, TestLab))
        # fit the model using the training set and validate on the subject data left out
        PredLab = model.predict(TestData, verbose = 0)

        # organlize testing and predicting labels into a dataframe that mirrors the order of original data arrays
        TestLabs = TestLabs + list(TestLab)
        PredLabs = PredLabs + list(PredLab)
        testInds = testInds + list(testInd)
            
    TestPred = pd.DataFrame(index = testInds)
    TestPred['TestLabs'] = TestLabs
    TestPred['PredLabs'] = PredLabs

    if segment:
        # iterate through each recording with clips that the neural network predicted on
        for rec in set(recordings):
            # get the score/label for the recording
            recLab = int(Labels[pd.Series(recordings) == rec][0])
            # get the subject corresponding to the recording
            recsub = int(subjects[pd.Series(recordings) == rec][0])
            # isolate the clip true and predicted labels that corresponding to the recording
            recTestPred = TestPred[pd.Series(recordings) == rec]
            
            if binary:
                if any(TestPred.PredLabs == 1):

                    if TestPred.PredLabs.value_counts()[1] > 1:

                        Recordings = Recordings + [rec]
                        Subjects = Subjects + [recsub]
                        Scores = Scores + [recLab]
                        Predictions = Predictions + [1]
                        # density is the relative amount of symptomatic clips in the recording
                        Densities = Densities + [TestPred.PredLabs.value_counts()[1] / 
                                             len(TestPred.PredLabs)]
                    # if not enough of the clips of the recording are scored as symptomatic - recording is normal
                    else:
                        Recordings = Recordings + [rec]
                        Subjects = Subjects + [recsub]
                        Scores = Scores + [recLab]
                        Predictions = Predictions + [0]
                        # density is the relative amount of symptomatic clips in the recording
                        Densities = Densities + [0]
                # if none of the clips are predicted symptomatic - recording is normal (0)
                else:
                    Recordings = Recordings + [rec]
                    Subjects = Subjects + [recsub]
                    Scores = Scores + [recsub]
                    Predictions = Predictions + [0]
                    # density is the relative amount of symptomatic clips in the recording
                    Densities = Densities + [0]
            # classify recording based on the maximum score given to a clip from it
            else:
                Recordings = Recordings + [rec]
                Subjects = Subjects + [recsub]
                Scores = Scores + [recSub]
                Predictions = Predictions + [np.max(TestPred.PredLabs)]
                # density is the relative amount of symptomatic clips in the recording
                # if statement avoids error if no zero scores are present
                if any(TSVTestPred.PredLabs == 0):
                    Densities = Densities + [1 - (TestPred.PredLabs.value_counts()[0] / len(TestPred.PredLabs))]
                else:
                    Densities = Densities + [1]
                        
    # consolidate predictions on recording scale with other metadata into a dataframe
    RecordingAnalysis = pd.DataFrame({'Subject': Subjects, 
                                      'Score': Scores, 'Prediction': Predictions, 'Density': Densities}, 
                                     index = Recordings)
    
    # print the overall accuracy of the full scope of this model in predicting recording scores
    print('Model Accuracy = ' + str((len(RecordingAnalysis[RecordingAnalysis.Score == RecordingAnalysis.Prediction])
                                     / len(RecordingAnalysis)) * 100))
    
    # format predictions for use in a confusion matrix that will be outputted
    CM = sklearn.metrics.confusion_matrix(RecordingAnalysis.Score.values, RecordingAnalysis.Prediction.values, 
                                          labels = le.classes_)
    
    # normalize confusion matrix rows to percents
    for i in range(len(CM)):
        CM[i, :] = (CM[i, :] / sum(CM[i, :])) * 100
        
    if binary:

# -------------------------------------------------------------------------------------------------------------------------
        
        # plot ROC curve for each subject whose data was used to validate the fit model for analysis
        plt.figure(figsize = (12, 10))
        for subject in RecordingAnalysis.Subject.unique():
            SRecordingAnalysis = RecordingAnalysis[RecordingAnalysis.Subject == subject]
            if (len(SRecordingAnalysis) == 1):
                continue
            # true labels: scored binary labels of each recording
            TL = SRecordingAnalysis.Score.values
            if all(TL == 0):
                continue
            # predicted labels: density of symptomatic clips in symptomatic recording
            PL = SRecordingAnalysis.Density.values
            # generate ROC curve
            fpr, tpr, thresholds = sklearn.metrics.roc_curve(TL, PL)
            plt.plot(fpr, tpr)
            plt.xlabel('False Positive Rate', fontsize = 20)
            plt.ylabel('True Positive Rate', fontsize = 20)
        # generate legend of subjects that correspond to curves
        plt.legend(RecordingAnalysis.Subject.unique())
        plt.show()
            
# -------------------------------------------------------------------------------------------------------------------------

        plt.figure(figsize = (6, 5))
    else:
        plt.figure(figsize = (12, 10))
    # plot confusion matrix to analyze true and predicted labels of recordings
    sns.heatmap(CM, vmin = 0, vmax = 100, annot = True)
    plt.xticks(fontsize = 14)
    if binary:
        plt.yticks(fontsize = 14)
    else:
        plt.yticks(fontsize = 14, rotation = 0)
    plt.xlabel('Predicted', fontsize = 16)
    plt.ylabel('Test', fontsize = 16)
    
    return RecordingAnalysis

In [9]:
RecordingAnalysis = DetectDiscernAnomalyNN(['Sitng', 'Stndg'], True, True, True, metaData)

(Samples, Timesteps, Features (Axes)) = (3053, 500, 3)
Labels = 3053
Train on 968 samples, validate on 81 samples
Epoch 1/10
968/968 [==============================] - 14s 14ms/step - loss: 0.4863 - val_loss: 0.4831
Epoch 2/10
968/968 [==============================] - 15s 16ms/step - loss: 0.4432 - val_loss: 0.4723
Epoch 3/10
968/968 [==============================] - 12s 12ms/step - loss: 0.3723 - val_loss: 0.5345
Epoch 4/10
968/968 [==============================] - 12s 12ms/step - loss: 0.3338 - val_loss: 0.5821
Epoch 5/10
968/968 [==============================] - 12s 12ms/step - loss: 0.3584 - val_loss: 0.6008
Epoch 6/10
968/968 [==============================] - 12s 12ms/step - loss: 0.3555 - val_loss: 0.6024
Epoch 7/10
968/968 [==============================] - 12s 12ms/step - loss: 0.3499 - val_loss: 0.5900
Epoch 8/10
968/968 [==============================] - 12s 12ms/step - loss: 0.3362 - val_loss: 0.6037
Epoch 9/10
968/968 [==============================] - 12s 12ms/step - 

Epoch 6/10
1018/1018 [==============================] - 11s 11ms/step - loss: 0.3432 - val_loss: 0.5481
Epoch 7/10
1018/1018 [==============================] - 12s 11ms/step - loss: 0.3176 - val_loss: 0.5481
Epoch 8/10
1018/1018 [==============================] - 12s 12ms/step - loss: 0.3492 - val_loss: 0.5482
Epoch 9/10
1018/1018 [==============================] - 12s 11ms/step - loss: 0.3537 - val_loss: 0.5482
Epoch 10/10
1018/1018 [==============================] - 11s 11ms/step - loss: 0.3586 - val_loss: 0.5482
Train on 1029 samples, validate on 20 samples
Epoch 1/10
1029/1029 [==============================] - 13s 12ms/step - loss: 0.4828 - val_loss: 0.5819
Epoch 2/10
1029/1029 [==============================] - 11s 11ms/step - loss: 0.4323 - val_loss: 0.5900
Epoch 3/10
1029/1029 [==============================] - 11s 11ms/step - loss: 0.3882 - val_loss: 0.5323
Epoch 4/10
1029/1029 [==============================] - 11s 11ms/step - loss: 0.3544 - val_loss: 0.4724
Epoch 5/10
1029/1

1044/1044 [==============================] - 13s 13ms/step - loss: 0.4932 - val_loss: 0.3443
Epoch 2/10
1044/1044 [==============================] - 12s 11ms/step - loss: 0.4558 - val_loss: 0.0438
Epoch 3/10
1044/1044 [==============================] - 12s 11ms/step - loss: 0.4401 - val_loss: 0.0114
Epoch 4/10
1044/1044 [==============================] - 12s 11ms/step - loss: 0.4294 - val_loss: 0.0064
Epoch 5/10
1044/1044 [==============================] - 11s 11ms/step - loss: 0.3829 - val_loss: 0.0053
Epoch 6/10
1044/1044 [==============================] - 12s 11ms/step - loss: 0.3840 - val_loss: 0.0052
Epoch 7/10
1044/1044 [==============================] - 12s 11ms/step - loss: 0.3842 - val_loss: 0.0046
Epoch 8/10
1044/1044 [==============================] - 12s 12ms/step - loss: 0.3826 - val_loss: 0.0037
Epoch 9/10
1044/1044 [==============================] - 12s 11ms/step - loss: 0.3822 - val_loss: 0.0031
Epoch 10/10
1044/1044 [==============================] - 11s 11ms/step - lo

Epoch 8/10
986/986 [==============================] - 12s 12ms/step - loss: 0.4025 - val_loss: 0.1454
Epoch 9/10
986/986 [==============================] - 12s 12ms/step - loss: 0.4023 - val_loss: 0.1448
Epoch 10/10
986/986 [==============================] - 12s 12ms/step - loss: 0.4032 - val_loss: 0.1445


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U12') dtype('<U12') dtype('<U12')